In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
# model.models[1].model_FlatVel = seis_model.DummyModel()
# model.models[1].model_FlatVel.state=1
# model.models[1].model_Style_A = seis_model.DummyModel()
# model.models[1].model_Style_A.state=1
# model.models[1].model_TV2D = seis_model.DummyModel()
# model.models[1].model_TV2D.state=1
model.read_cache = False

In [3]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::500]
model.models[1].run_in_parallel = True
model.models[1].model_Style_A.models = model.models[1].model_Style_A.models[0:1]
data_out = model.infer(data)

Processing in parallel : 100%|█████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.38s/it]


CPU times: total: 2.05 s
Wall time: 20.2 s


In [4]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')

In [5]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    try:
        res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    except:
        res['time_taken'].append(0)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df



Brendan local score:  21.10849830098055
My local score:  7.931421027981505
Estimate public LB:  10.82146737057351


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,4.223399,4.223399,4.223399,0.428802
1,CurveFault_B,53.323469,12.834694,12.834694,0.626699
2,CurveVel_A,11.058163,2.745102,2.745102,0.302696
3,CurveVel_B,35.980816,11.154490,11.154490,0.378848
4,FlatFault_A,2.156559,2.156559,2.156559,0.171584
5,FlatFault_B,13.678367,5.901020,5.901020,0.353245
6,FlatVel_A,1.304286,0.014286,0.014286,0.281469
7,FlatVel_B,0.697347,0.000000,0.000000,0.143508
8,Style_A,29.772975,3.052164,3.052164,0.159782
9,Style_B,58.889601,37.232496,37.232496,0.152753


In [6]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [7]:
28-30*0.081

25.57

In [8]:
65000*0.09*0.15/4

219.375

In [9]:
8

8